In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import random
from agent import Agent
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
env = UnityEnvironment(file_name='Env_1_Agent/Reacher_Linux/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [5]:
"""
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
"""

"\nenv_info = env.reset(train_mode=False)[brain_name]     # reset the environment    \nstates = env_info.vector_observations                  # get the current state (for each agent)\nscores = np.zeros(num_agents)                          # initialize the score (for each agent)\nwhile True:\n    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)\n    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1\n    env_info = env.step(actions)[brain_name]           # send all actions to tne environment\n    next_states = env_info.vector_observations         # get next state (for each agent)\n    rewards = env_info.rewards                         # get reward (for each agent)\n    dones = env_info.local_done                        # see if episode finished\n    scores += env_info.rewards                         # update the score (for each agent)\n    states = next_states                               # roll over states to next ti

In [6]:
#definition hyperparameters and trainingsconditions
n_episodes = 150
max_eps= 1.0
min_eps = 0.01
eps_decay = 0.95
training_mode = True
random_seed = 0

In [ ]:
# Initialize Feed-forward DNNs for Actor and Critic models. 
agent = Agent(state_size, action_size, random_seed, max_eps, eps_decay)

#train the agent
def ddpg_train(n_episodes):

    #list containing scores from each episode
    scores = []
    #last 100 scores
    scores_window = deque(maxlen=100)
    
    for episode in range(n_episodes): 
        # Reset the enviroment
        env_info = env.reset(train_mode=training_mode)[brain_name] 
        cur_states = env_info.vector_observations
        score = np.zeros(num_agents)
        # initialize timestep
        timestep = 0
        # reset noise
        agent.reset()
        
        while True:
            
            # Predict the action for the current state.
            # Achtung Epsilon entspricht noch nicht dem wahren Epsilon
            """
            if random.random() < eps:
                # Choose random action
                action = np.random.randn(num_agents, action_size)
                action = np.clip(action, -1, 1)
            else:
            """
            # Choose best action for given network
            actions = agent.act(cur_states, add_noise = True)
        
            # Action is performed and new state, reward, info are received. 
            env_info = env.step(actions)[brain_name]
        
            # get next state 
            next_states = env_info.vector_observations
            
            # see if episode is finished
            dones = env_info.local_done
            
            # get reward
            rewards = env_info.rewards
            
            # current state, action, reward, new state are stored in the experience replay
            agent.step(cur_states, actions, rewards, next_states, dones, timestep)
            """
            # save experience to replay buffer, perform learning step at defined interval
            for cur_state, action, reward, next_state, done in zip(cur_states, actions, rewards, next_states, dones):
                
                # current state, action, reward, new state are stored in the experience replay
                agent.step(cur_state, action, reward, next_state, done, timestep)
            """
            # roll over new state
            cur_states = next_states
            
            #add reward to score
            score += rewards
            
            timestep+=1
            
            if np.any(dones):
                break
        
        print("Episode:" + str(episode) + " Score:" + str(score))
        """   
        #decrease epsilon
        eps = max(eps_end, eps_decay*eps)
        """
        #save most recent score
        scores_window.append(score)
        scores.append(score)
        
        # save model weights and print mean score over last 100 episodes
        if (episode+1) % 100 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_Actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_Critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode+1, np.mean(scores_window)))
    
    return scores

scores = ddpg_train(n_episodes)

#plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode')
plt.show()

Episode:0 Score:[0.77999998]
Episode:1 Score:[3.25999993]
Episode:2 Score:[0.83999998]
Episode:3 Score:[2.11999995]
Episode:4 Score:[0.96999998]
Episode:5 Score:[1.39999997]
Episode:6 Score:[0.90999998]
Episode:7 Score:[0.89999998]
Episode:8 Score:[2.03999995]
Episode:9 Score:[1.00999998]
Episode:10 Score:[0.80999998]
Episode:11 Score:[0.93999998]
Episode:12 Score:[0.33999999]
Episode:13 Score:[0.80999998]
Episode:14 Score:[0.]
Episode:15 Score:[2.50999994]
Episode:16 Score:[0.80999998]
Episode:17 Score:[2.48999994]
Episode:18 Score:[1.84999996]
Episode:19 Score:[0.81999998]
Episode:20 Score:[1.10999998]
Episode:21 Score:[1.64999996]
Episode:22 Score:[0.64999999]
Episode:23 Score:[0.94999998]
Episode:24 Score:[1.39999997]
Episode:25 Score:[1.06999998]
Episode:26 Score:[1.15999997]
Episode:27 Score:[0.77999998]
Episode:28 Score:[0.71999998]
Episode:29 Score:[0.50999999]
Episode:30 Score:[0.81999998]
Episode:31 Score:[0.67999998]
Episode:32 Score:[2.49999994]
Episode:33 Score:[1.23999997

In [ ]:
env.close()